In [ ]:
import numpy as np
import pandas as pd
import DataProcessing as dp
import Arima
import matplotlib.pyplot as plt

In [ ]:
parkdata = pd.read_parquet('Data/archive/est_hourly.paruqet')
parkdata.drop(['NI','PJM_Load'], inplace=True, axis = 1)
parkdata = parkdata.sort_index()
filled_data = dp.fill_empty(parkdata, debug=False)
filled_data_feat = dp.feature_creation(filled_data,['AEP'], roll_amt=4)
filled_data_feat = dp.fill_empty(filled_data_feat, debug=False)
temp = filled_data_feat.copy()
# print(filled_data_feat.columns)

In [ ]:
x_diff, x_hat, residuals, d, p, q, phi, theta = Arima.prediction_engine(temp['AEP'],100)
Arima.save_data(x_diff, x_hat, residuals, d, p, q, phi, theta)

In [ ]:
forecast_horizon = 24
forecasted_x = Arima.forecast(x_diff,p,q,forecast_horizon,phi,theta,residuals)
forecasted_x = Arima.build_forecast_dataframe(forecasted_x, temp['AEP'], d, forecast_horizon, include_inverse=False)

plt.plot(x_diff.tail(200))
plt.plot(forecasted_x)
plt.show()